In [ ]:
import base64
import uuid

from freezegun import freeze_time
from registry_schemas.example_data import (
    ADDRESS,
    ALTERATION,
    ANNUAL_REPORT,
    BUSINESS,
    CHANGE_OF_ADDRESS,
    CHANGE_OF_DIRECTORS,
    COMMENT_BUSINESS,
    COMMENT_FILING,
#     CONTACT_POINT,
    CORRECTION_AR,
    CONVERSION,
#     COOP_INCORPORATION,
    COURT_ORDER,
    DISSOLUTION,
    FILING_HEADER,
    INCORPORATION,
    REGISTRARS_NOTATION,
    REGISTRARS_ORDER,
#     RESTORATION,
    SPECIAL_RESOLUTION,
    TRANSITION,
)
from sqlalchemy_continuum import versioning_manager

from legal_api.models import (
    Address,
    Alias,
    Business,
    Comment,
    Filing,
    Office,
    Party,
    PartyRole,
    ShareClass,
    ShareSeries,
    User,
    db,
)
from legal_api.models.colin_event_id import ColinEventId
from legal_api.utils.datetime import datetime, timezone

In [1]:
def factory_completed_filing(business, data_dict, filing_date, payment_token=None, colin_id=None):
    """Create a completed filing."""
    if not payment_token:
        payment_token = str(base64.urlsafe_b64encode(uuid.uuid4().bytes)).replace('=', '')

    with freeze_time(filing_date):

        filing = Filing()
        filing.business_id = business.id
        filing.filing_date = filing_date
        filing.filing_json = data_dict
        filing.save()

        uow = versioning_manager.unit_of_work(db.session)
        transaction = uow.create_transaction(db.session)
        filing.transaction_id = transaction.id
        filing.payment_token = payment_token
        filing.effective_date = filing_date
        filing.payment_completion_date = filing_date
        if colin_id:
            colin_event = ColinEventId()
            colin_event.colin_event_id = colin_id
            colin_event.filing_id = filing.id
            colin_event.save()
        filing.save()
    return filing

In [ ]:
def factory_pending_filing(business, data_dict, filing_date, payment_token=None, colin_id=None):
    """Create a completed filing."""
    if not payment_token:
        payment_token = str(base64.urlsafe_b64encode(uuid.uuid4().bytes)).replace('=', '')

    with freeze_time(filing_date):

        filing = Filing()
        filing.business_id = business.id
        filing.filing_date = filing_date
        filing.filing_json = data_dict
        filing.save()

        filing.payment_token = payment_token
        
        filing._status = Filing.Status.PENDING

        filing.save()
    return filing

In [ ]:
def factory_pending_IA_filing(bootstrap, data_dict, filing_date, payment_token=None, colin_id=None):
    """Create a completed filing."""
    if not payment_token:
        payment_token = str(base64.urlsafe_b64encode(uuid.uuid4().bytes)).replace('=', '')

    with freeze_time(filing_date):

        filing = Filing()
        filing.temp_reg = bootstrap.identifier
        filing.filing_date = filing_date
        filing.filing_json = data_dict
        filing.save()

        filing.payment_token = payment_token
        
        filing._status = Filing.Status.PENDING

        filing.save()
    return filing